# Zoning Code Compliance Checker

This Jupyter notebook demonstrates a reproducible workflow for checking parcel-level compliance
with zoning regulations (FAR, BCR, setbacks, height limits) using modern Python libraries.


In [ ]:
import geopandas as gpd
import pandas as pd
import json
from shapely.validation import make_valid
from pyproj import CRS

# Helper to compute UTM CRS
def get_utm_crs(gdf):
    centroid = gdf.geometry.unary_union.centroid
    lon, lat = centroid.x, centroid.y
    utm_zone = int((lon + 180) / 6) + 1
    return CRS.from_dict({ "proj": "utm", "zone": utm_zone, "south": lat < 0 })

# Helper to clean invalid geometries
def clean_geom(g):
    if not g.is_valid:
        try:
            return make_valid(g)
        except Exception:
            return g.buffer(0)
    return g


In [ ]:
# Load parcel, building, and zoning data
parcels = gpd.read_file("data/parcels.geojson").to_crs("EPSG:4326")
buildings = gpd.read_file("data/buildings.geojson").to_crs("EPSG:4326")
zoning = gpd.read_file("data/zoning.geojson").to_crs("EPSG:4326")

# Load zoning rules
with open("data/zoning_rules.json") as f:
    zoning_rules = json.load(f)


In [ ]:
# Join parcels to zoning districts
parcels = gpd.sjoin(parcels, zoning[['zone_id', 'geometry']], how='left', predicate='intersects')

# Extract building footprints per parcel
buildings = gpd.sjoin(buildings, parcels[['parcel_id','geometry']], how='left', predicate='within')

# Compute Floor Area Ratio (FAR)
buildings['floor_area'] = buildings.geometry.area * buildings['num_floors']
fa = buildings.groupby('parcel_id')['floor_area'].sum()
parcels['floor_area'] = parcels['parcel_id'].map(fa)
parcels['FAR'] = parcels['floor_area'] / parcels.geometry.area

# Flag FAR violations
parcels['FAR_violation'] = parcels.apply(lambda r: r['FAR'] > zoning_rules.get(r['zone_id'], {}).get('FAR', 0), axis=1)


In [ ]:
# Export violations to GeoJSON
violations = parcels[parcels['FAR_violation']].copy()
violations.to_file('output/far_violations.geojson', driver='GeoJSON')

# Display summary
import matplotlib.pyplot as plt
parcels['FAR_violation'].value_counts().plot(kind='bar', title='FAR Violations')
plt.show()
